# Ligand Selection and Preparation, Docking and Analysis

This notebook contains the code necessary to; prepare the ligands for docking, run docking in smina, and analyse the results of the docking.

## Ligand Selection, Validation and Standardisation

Ligand selection step involves;

- Identification of ligands (from generative model, DB's, etc) and extraction as smiles strings
- Standardisation - smiles-->mol, mol--> smiles, removal of any ligands that have no smiles
- Filtering of ligands (based on pharmacophore matching score, eos models, synthetic likelihood, etc)

In [6]:
# Ligand standardisation code here - smiles -> mol -> smiles



## Ligand Preparation

- Conversion of smiles strings to 3D conformers using RDKIT 
- Protonation at a specific pH (7.4) and conversion to .pdbqt via obabel
- OR
- Protonation at a specific pH, conversion to .sdf and merging of all .sdf files into one .sdf file containing all ligands

In [25]:
# Ensure docking-env is activated as kernel.

# Import all libraries that are required.

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdDistGeom
from rdkit.Chem.rdmolfiles import MolToPDBFile


In [26]:
import os

DATAPATH = '/data'
RESULTSPATH = '../results'

In [29]:
# The below takes a .csv file with smiles strings and converts them all to individual .pdbqt files.

# Update doc string
def prepare_ligands(csv_file_name,  pH, header_len=1, output_dir='', delim=',') -> list:

    """
    Takes a csv file of smiles, generates 3D coordinates,
    protonates for a specific pH, and outputs a pdbqt file 
    for each of the compounds therein. 

      
    csv_file_name (str): File path to a csv file containing the smiles input, among other information.
    
    header_len (int): num of rows to skip in the csv file.
    
    pH (float): pH that the compounds will be protonated at.
    
    output_dir (str): str appended to the produced file name so output files can be conveniently stored in an
                        output directory
    
    delim (str): delimeter used in the csv file

        
    returns: list of strs of the produced pdbqt file's paths.
    
    """
    
    out_pdbqts = [] # Creates an empty list to fill.
    
    print(csv_file_name)
    
    with open(csv_file_name, 'r') as csv: 
        
        
        for entry in csv.readlines()[header_len:]:
            
            Similarity, ID, SMILES = entry.split(delim)[:3]  # Note that this must be changed depending on the headings in the .csv file.
            
            # Convert smiles str to 3D coordinates
            mol = Chem.MolFromSmiles(SMILES)
            mol = Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol)
            
    
            # Ouput coords to pdb
            pdb_name = f"{output_dir}/{ID}.pdb"
            MolToPDBFile(mol, pdb_name)
            
#             print(pdb_name)
            # Protonate according to pH
            ! obabel {pdb_name} -pH {pH} -O {pdb_name}
            
            
            # Also create a pdbqt for vina
            pdbqt_str = pdb_name + 'qt'
            ! obabel {pdb_name} -pH {pH} -O {pdbqt_str}
            
            
            out_pdbqts.append(pdbqt_str)
#             print()
            
    return out_pdbqts
        
        
        

In [32]:
pH = 7.4

In [ ]:
# myfile = os.path.join(DATAPATH, "sim29", "smiles3.csv") -- haven't included this step, as haven't worked out datapth inclusion yet.

ligands = prepare_ligands('../data/sim29/smiles3.csv', pH, output_dir='../data/sim29')

#obabel_command = "obabel " + preparation_output_filename + " -osdf -O " + conformer_output_filename + " --conformer --nconf 1 --score energy --weighted"



In [31]:
# The below takes a .csv file with smiles strings and converts them to a single .sdf file containing all smiles strings in protonated 3D format.

# Update doc string
def prepare_ligands_sdf(csv_file_name,  pH, header_len=1, output_dir='', delim=',') -> list:

    """
    Takes a csv file of smiles, generates 3D coordinates,
    protonates for a specific pH, and outputs a pdbqt file 
    for each of the compounds therein. 

      
    csv_file_name (str): File path to a csv file containing the smiles input, among other information.
    
    header_len (int): num of rows to skip in the csv file.
    
    pH (float): pH that the compounds will be protonated at.
    
    output_dir (str): str appended to the produced file name so output files can be conveniently stored in an
                        output directory
    
    delim (str): delimeter used in the csv file

        
    returns: list of strs of the produced pdbqt file's paths.
    
    """
    
    out_sdfs = [] # Creates an empty list to fill.
    
    print(csv_file_name)
    
    with open(csv_file_name, 'r') as csv: 
        
        
        for entry in csv.readlines()[header_len:]:
            
            Similarity, ID, SMILES = entry.split(delim)[:3]  # Note that this must be changed depending on the headings in the .csv file.
            
            # Convert smiles str to 3D coordinates
            mol = Chem.MolFromSmiles(SMILES)
            mol = Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol)
            
    
            # Ouput coords to pdb
            pdb_name = f"{output_dir}/{ID}.pdb"
            MolToPDBFile(mol, pdb_name)
            
#             print(pdb_name)
            # Protonate according to pH, convert to .sdf
            sdf_name = f"{output_dir}/{ID}.sdf"
            ! obabel {pdb_name} -pH {pH} -O {sdf_name}
            
            os.remove(pdb_name) # removes the .pdb files after obabel protonates and converts to .sdf
            
            out_sdfs.append(sdf_name)
#             print()
            
    return out_sdfs



def merge_sdfs(out_sdfs, merged_sdf):

    mols = []
    for out_sdf in out_sdfs:
        suppl = Chem.SDMolSupplier(out_sdf)
        for mol in suppl:
            mols += [mol]
        os.remove(out_sdf)

    with Chem.SDWriter(merged_sdf) as w:
        for mol in mols:
            w.write(mol)

def prepare_and_merge_ligands(csv_file_name,  pH, header_len=1, output_dir='', delim=','):
    ligands = prepare_ligands_sdf(csv_file_name,  pH, header_len, output_dir, delim)
    merge_sdfs(ligands, output_dir+"/merged.sdf")
        

In [ ]:
prepare_and_merge_ligands('../data/sim29_sdf/smiles3.csv', pH, output_dir='../data/sim29_sdf')

# It is noteworthy that this function, that produces one file, is quicker than the previous function.


In [1]:
# The below is the same as the above, with **multiple conformers** generated.

# Update doc string
def prepare_multiple_conf_ligands_sdf(csv_file_name,  pH, header_len=1, output_dir='', delim=',') -> list:

    """
    Takes a csv file of smiles, generates 3D coordinates,
    protonates for a specific pH, and outputs a pdbqt file 
    for each of the compounds therein. 

      
    csv_file_name (str): File path to a csv file containing the smiles input, among other information.
    
    header_len (int): num of rows to skip in the csv file.
    
    pH (float): pH that the compounds will be protonated at.
    
    output_dir (str): str appended to the produced file name so output files can be conveniently stored in an
                        output directory
    
    delim (str): delimeter used in the csv file

        
    returns: list of strs of the produced pdbqt file's paths.
    
    """
    
    out_sdfs = [] # Creates an empty list to fill.
    
    print(csv_file_name)
    
    with open(csv_file_name, 'r') as csv: 
        
        
        for entry in csv.readlines()[header_len:]:
            
            Similarity, ID, SMILES = entry.split(delim)[:3]  # Note that this must be changed depending on the headings in the .csv file.
            
            # Convert smiles str to 3D coordinates
            mol = Chem.MolFromSmiles(SMILES)
            mol = Chem.AddHs(mol)
            Chem.rdDistGeom.EmbedMultipleConfs(mol, numConfs=5, pruneRmsThresh=1.0)
            
    
            # Ouput coords to pdb
            pdb_name = f"{output_dir}/{ID}.pdb"
            MolToPDBFile(mol, pdb_name)
            
#             print(pdb_name)
            # Protonate according to pH, convert to .sdf
            sdf_name = f"{output_dir}/{ID}.sdf"
            ! obabel {pdb_name} -pH {pH} -O {sdf_name}
            
            os.remove(pdb_name) # removes the .pdb files after obabel protonates and converts to .sdf
            
            out_sdfs.append(sdf_name)
#             print()
            
    return out_sdfs



def merge_sdfs(out_sdfs, merged_sdf):

    mols = []
    for out_sdf in out_sdfs:
        suppl = Chem.SDMolSupplier(out_sdf)
        for mol in suppl:
            mols += [mol]
        os.remove(out_sdf)

    with Chem.SDWriter(merged_sdf) as w:
        for mol in mols:
            w.write(mol)

def prepare_and_merge_multipleconf_ligands(csv_file_name,  pH, header_len=1, output_dir='', delim=','):
    ligands = prepare_multiple_conf_ligands_sdf(csv_file_name,  pH, header_len, output_dir, delim)
    merge_sdfs(ligands, output_dir+"/merged_multipleconfs.sdf")
        

In [ ]:
prepare_and_merge_multipleconf_ligands('../data/sim29_sdf/smiles3.csv', pH, output_dir='../data/sim29_sdf')

# Docking with smina

- Receptor is prepped manually for this project using ADTools (waters removed, polar hydrogens added, Gasteiger charges added, saved as .pdbqt file)
- Flexible side chains identified (not done below...)
- Ligand (CHO) can be readded as hetatm/different chain for autobox generation
- smina run (search space defined on command line) and outputs saved

The below is for 1 ligand, previously prepared in notebook _0:

In [ ]:
# This is for a single compound, previously defined... I think I will just add the abyssomicin C smiles etc to my sim29 list for ease and delete this cell.

receptor = '../data/protein/pabb_model1.pdbqt'  # Receptor
ligand = '../data/abyc/abyC3d.pdbqt'  # Ligand
log = '..results/abyc/output_log.txt'
path_to_results = '../results/abyc/outputs_addHoff.sdf'
path_to_smina = '../src/smina.static'

cmd = path_to_smina  + " -r " + receptor + " -l " + ligand + " -o " + path_to_results + " --log " + log + " --seed 42" + " --center_x 74 " + " --center_y 47 " + " --center_z 57 " + " --size_x 25 " + " --size_y 25 " + " --size_z 32 " + " --exhaustiveness 1000 " + " --num_modes 10 " + " --addH off"

os.system(cmd)

In [ ]:
# This is my attempt to run all pdbqt file from sim29 - this doesn't work, but I don't think it's necessary...

# Think I need to define all of the .pdbqt files in the folder:
'''   from pathlib import Path

mydir = Path("path/to/my/dir")
for file in mydir.glob('*.mp4'):
    print(file.name)
    # do your stuff
    '''

receptor = '../data/protein/pabb_model1.pdbqt'  # Receptor
ligands = '../data/sim29/*.pdbqt'  # Ligand
log = '../results/sim29/log_multifile_pdbqt.txt' # log file
path_to_results = '../results/sim29/outputs_multifile_pdbqt.sdf'
path_to_smina = '../src/smina.static'

cmd = path_to_smina  + " -r " + receptor + " -l " + ligands + " -o " + path_to_results + " --log " + log + " --seed 42" + " --center_x 74 " + " --center_y 47 " + " --center_z 57 " + " --size_x 25 " + " --size_y 25 " + " --size_z 32 " + " --exhaustiveness 25 " + " --num_modes 1 " + " --addH off "

os.system(cmd)

In [ ]:
# The below is smina, run using the single .sdf file with all ligands. 

receptor = '../data/protein/pabb_model1.pdbqt'  # Receptor
ligands = '../data/sim29_sdf/merged.sdf'  # Ligand
log = '../results/sim29/log_sglfile_sdf.txt' # log file
path_to_results = '../results/sim29/outputs_sgl_file_sdf.sdf' 
path_to_smina = '../src/smina.static'

cmd = path_to_smina  + " -r " + receptor + " -l " + ligands + " -o " + path_to_results + " --log " + log + " --seed 42" + " --center_x 74 " + " --center_y 47 " + " --center_z 57 " + " --size_x 25 " + " --size_y 25 " + " --size_z 32 " + " --exhaustiveness 100 " + " --num_modes 1 " + " --addH off "

os.system(cmd)

In [ ]:
# The below is smina, run using the single .sdf file with all ligands, and multiple conformers per ligand. 

receptor = '../data/protein/pabb_model1.pdbqt'  # Receptor
ligands = '../data/sim29_sdf/merged_multipleconfs.sdf'  # Ligand
log = '../results/sim29/log_sglfile_multipleconfs_sdf.txt' # log file
path_to_results = '../results/sim29/outputs_multipleconfs_sdf.sdf' 
path_to_smina = '../src/smina.static'

cmd = path_to_smina  + " -r " + receptor + " -l " + ligands + " -o " + path_to_results + " --log " + log + " --seed 42" + " --center_x 74 " + " --center_y 47 " + " --center_z 57 " + " --size_x 25 " + " --size_y 25 " + " --size_z 32 " + " --exhaustiveness 20 " + " --num_modes 1 " + " --addH off "

os.system(cmd)

## Analysis

Here, the top poses will be selected and analysed.

- See SDSorter, with 'reduceconf' and 'nbestx'
- Also see 'sminalog_analysis.py'

In [12]:
# The below is the sminalog_analysis.py fron bioinformatics review bitbucket.

''' import os
import itertools
import collections
import pprint
import sys
import os.path
import glob
import re

mypath = os.path.abspath(os.getcwd())                                           # get path of current dir

print("Directory path detected : ", mypath)

logfile = glob.glob('*log*.txt')                                #getting the log filename
print(logfile)
file_path = os.path.join(mypath, logfile[0])


print("\nFile path detected : ", file_path)

with open(logfile[0], "r") as f:
    i = 0;
    for line in f:
        if '-+' in line:
            nextline = next(f)
            i = i + 1

            nextlinearray  = nextline.split()                       #splitting the first row in different values
            bind_aff = nextlinearray[1]                             #getting the binding affinity of first pose

            with open("output.txt", "a") as myfile:
                 print(i, " : ", bind_aff, end='\n', file=myfile)


    print("Done! The result is provided in the output.txt file.")
    '''



sdsorter - sort minimizedAffinity - reduceconfs 1 2 WEI _ docked . sdf . gz \
- print -c > 2 WEI _ docked . txt


' import os\nimport itertools\nimport collections\nimport pprint\nimport sys\nimport os.path\nimport glob\nimport re\n\nmypath = os.path.abspath(os.getcwd())                                           # get path of current dir\n\nprint("Directory path detected : ", mypath)\n\nlogfile = glob.glob(\'*log*.txt\')                                #getting the log filename\nprint(logfile)\nfile_path = os.path.join(mypath, logfile[0])\n\n\nprint("\nFile path detected : ", file_path)\n\nwith open(logfile[0], "r") as f:\n    i = 0;\n    for line in f:\n        if \'-+\' in line:\n            nextline = next(f)\n            i = i + 1\n\n            nextlinearray  = nextline.split()                       #splitting the first row in different values\n            bind_aff = nextlinearray[1]                             #getting the binding affinity of first pose\n\n            with open("output.txt", "a") as myfile:\n                 print(i, " : ", bind_aff, end=\'\n\', file=myfile)\n\n\n    print("D